In [1]:
import pandas as pd
import numpy as np
import graphlab as gl

In [2]:
def read_labels():
    apk_label_ds1_path = '../data/labels_vt_mal_2017_2020_az_2020_benign_hashed.csv'
    ds1_labels_df = pd.read_csv(apk_label_ds1_path)
    ds1_labels_df.head(2)
    print("Got data: ", len(ds1_labels_df))
    return ds1_labels_df

In [76]:
def transform_app_data(fname='../data/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.csv'):
    sf_ds1_full = gl.SFrame.read_csv(fname, header=False, verbose=False)
    _=sf_ds1_full.rename({'X3':'fcount'})
    sf_ds1_full['apk']=sf_ds1_full['X1'].apply(lambda x: x.upper())
    sf_ds1_full['function'] = sf_ds1_full['X2'].apply(lambda x: x.upper())
    
    _=sf_ds1_full.remove_columns(['X1', 'X2'])
    
    print("Unique APK:", len(set(sf_ds1_full['apk'].unique())))
    print("Unique functions:", len(set(sf_ds1_full['function'].unique())))

    outname = fname.replace('data', 'binarydata').replace('.csv', '.sframe')
    sf_ds1_full.save(outname, format='binary')

transform_app_data()

('Unique APK:', 10003)
('Unique functions:', 699854)


In [78]:
mw = gl.load_sframe('../binarydata/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.sframe')

In [79]:
mw.head()

fcount,apk,function
1,3407F2E5AB21E8966BDFAF33D805DE7AC97BED3A7454C1 ...,2D6877D28225C6E0F115275774CDA94C ...
1,006373DF53793664DDD73C7A96B18794A10DE6FAAD185B ...,4B673494945E9BF470675331FC1D6B27 ...
2,238230D0B795F97E596FA7379AF0769D74ED3810976092 ...,96D7A6B3A2BEA95CA0E3B74FE3C4E7F8 ...
1,25A3382B3FAAAC9D5A14FE8DE914D47B3BC6284599FD27 ...,A6D7D5D7968AE7BDC62AD696AE7E6BD1 ...
1,4D0F5074E2A7579DBC87C0D8483EFEB3D31B24E0008BD1 ...,20E0EB29F9F144A01276B87CFC0AC2A1 ...
1,94F2FB12A21085764C85DB0E9E4B381EEDD1DC65E4F2D7 ...,F66DB25FFCDEEB2D40CE702261BEEE83 ...
1,9AC8EDE1BF55180F0F90A8B127ACDF21AE9A8916FBE0E6 ...,B9AFD08C24947FE6E37A82672795427B ...
1,005CDFD24DEF5924C91E68DC0CF18D18B91D6118D4BE51 ...,4ABCBF64B57FAD928D0414890D41CC95 ...
1,0461C5356AF1B678D91A874041AAF9978764BC8A3E1833 ...,7D72BE96A185F93BD36B924811C8EEC5 ...
1,4A69FD5D71F3010B08B2C2E51CF942D2513D48ADD872C9 ...,4ABCBF64B57FAD928D0414890D41CC95 ...


In [9]:
benign_ds1 = ds1_labels_df[ds1_labels_df['malware_label'] == False]
print(len(benign_ds1))
malware_ds1 = ds1_labels_df[ds1_labels_df['malware_label'] == True]
print(len(malware_ds1))

9389
18329


In [ ]:
## jaccard
## gl.item_similarity_recommender.create(train_dataset, user_id='function', item_id='apk', only_top_k=k, 
#similarity_type='jaccard', target='fcount', verbose=False)

In [28]:
subsample = mw.sample(fraction=.02, seed=42)

In [ ]:
## sample: 
#benign_ds1_sample1 = benign_ds1.sample(5540, random_state=1)

In [32]:
reco = gl.item_similarity_recommender.create(subsample, user_id='function', item_id='apk', only_top_k=3, 
                                      similarity_type='jaccard', target='fcount', verbose=False)

Recsys training: model = item_similarity

In [45]:
smitems = reco.get_similar_items(verbose=False)

In [48]:
smitems.shape, subsample.shape

((20481, 4), (702716, 3))

In [36]:
#smitems.sort(sort_columns=)
smitems.column_names()

['apk', 'similar', 'score', 'rank']

In [43]:
smitems.sort(sort_columns='score', ascending=False)

apk,similar,score,rank
ABCAE14B324338373D5108C834D17DCF3AF123D195052C ...,CB1B7D16667C89337B14F597CDAFB4D3C0BD92511C83D1 ...,1.0,3
ABCAE14B324338373D5108C834D17DCF3AF123D195052C ...,0413FDBC2CDEBA91D85B2E91EEC46492DA769CCCA930FF ...,1.0,2
ABCAE14B324338373D5108C834D17DCF3AF123D195052C ...,E2522F34A731B441C6CA975FCC478D1EA7056022C46001 ...,1.0,1
89b773c6af0d8fa1ad0f4ebe5e015b3f14f273eb55ed89 ...,1c192f03de4e4c7f7b9f2e92a3e9d120cc73c82290bad4 ...,1.0,3
89b773c6af0d8fa1ad0f4ebe5e015b3f14f273eb55ed89 ...,a53e486b95a3d01fa5ccb4b5d4441c95b79778e040a2ad ...,1.0,2
89b773c6af0d8fa1ad0f4ebe5e015b3f14f273eb55ed89 ...,83f25cbbb6f528239e984892c38d68e1bc1cf4395a0cae ...,1.0,1
D28AC61FA68397D5792982EF393CD2B33DD84FEE20ABCD ...,1DBC5226CF759C06D3F618B05BD817C5D02523AE6705E0 ...,1.0,3
D28AC61FA68397D5792982EF393CD2B33DD84FEE20ABCD ...,C2E087F165E0B3A112356FA6033C1833250F60AB149CDD ...,1.0,2
D28AC61FA68397D5792982EF393CD2B33DD84FEE20ABCD ...,E446661738FD46A58905831D118A151EDF36C52AAE4D8E ...,1.0,1
6386b618cd5f18fcba5eaaf951ae09172d51d4f0da5872 ...,0ef8bb8c4285458248ae3638b7e8eb6328b72523178700 ...,1.0,1


## Gamma network 
as implemented by Tataian

In [80]:
mw = gl.load_sframe('../binarydata/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.sframe')

In [86]:
apks = mw['apk'].unique()

In [89]:
sample_apks = apks.sample(fraction=0.02, seed=42)

In [97]:
subsamp = mw.filter_by(sample_apks, column_name='apk')

In [98]:
len(subsamp['apk'].unique()), len(sample_apks)

(196, 196)

## Right position for $k$

it is not sufficient to set k for `get_similar_items`. The recomended performs a cut-off anyways. The $k$ has to be set in the `create` method (`only_top_k`). 

There is also a `threshold` in `create` initialization, but apparently this has less influece.

In [99]:
k = len(subsamp['apk'].unique())

In [220]:

sim_recom = gl.item_similarity_recommender.create(subsamp, 
                                                  user_id='function',
                                                  item_id='apk',
                                                  similarity_type='jaccard',
                                                  verbose=False, only_top_k=k)

Recsys training: model = item_similarity

In [104]:
apk1, apk2 = sample_apks[:2]

In [234]:
#| for union.& for intersection.– for difference^ for symmetric difference
#return 1 - len(p1 & p2)/len(p1|p2)

def get_funcs(tab, apk):
    return tab[tab['apk']==apk]['function']

def alt_jaccard(apk1, apk2, tab):
    ap1func = set(get_funcs(tab, apk1))
    ap2func = set(get_funcs(tab, apk2))
    
    return float(len(ap1func & ap2func)) / float(len(ap1func| ap2func))

In [218]:
def get_similar_apks(apk):
    similar_items = sim_recom.get_similar_items([apk], k=k)['similar', 'score']
    similar_items.materialize()
    return similar_items

def get_jaccard_sim(apk1, apk2):
    similar_to_apk1 = get_similar_apks(apk1)
    try:
        return similar_to_apk1[similar_to_apk1['similar']==apk2]['score'][0]
    except:
        return 0


### Precision issues

In [221]:
get_jaccard_sim(apk1, apk2), alt_jaccard(apk1, apk2, subsamp)

(0.002584218978881836, 0.002584194731577192)

In [223]:
ap1func = set(get_funcs(subsamp, apk1))
ap2func = set(get_funcs(subsamp, apk2))

len(ap1func & ap2func), len(ap1func | ap2func)

(31, 11996)

In [224]:
31.0 / 11996.0

0.002584194731577192

In [225]:
apk1 = sample_apks[0]

In [229]:
%%timeit
for apks in sample_apks[1:100]:
    dist = get_jaccard_sim(apk1=apk1, apk2=apks)

1 loop, best of 3: 603 ms per loop


## Notices
1. convert hashes to ints?
2. subsample


In [68]:
str.upper()

TypeError: descriptor 'upper' of 'str' object needs an argument